# Gradio UI

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
import gradio as gr

In [3]:
load_dotenv(override=True)
z_ai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if z_ai_api_key:
    print(f"Z.AI API Key exists and begins {z_ai_api_key[:8]}")
else:
    print("Z.AI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

Z.AI API Key exists and begins 616b9f52
Google API Key exists and begins AIzaSyDD


In [4]:
zai_url="https://api.z.ai/api/coding/paas/v4"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

zai = OpenAI(api_key=z_ai_api_key, base_url=zai_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

In [5]:
# Let's wrap a call to GLM-4.5-Air in a simple function

system_message = "You are a helpful assistant"

def message_glm(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = zai.chat.completions.create(model="GLM-4.5-Air", messages=messages)
    return response.choices[0].message.content

In [6]:
# This can reveal the "training cut off", or the most recent date in the training data

message_glm("What is today's date?")

"\nI don't have access to real-time information or the current date. You'll need to check your device's clock, calendar, or search online to find today's date."

## User Interface

In [7]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [8]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [9]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.


In [10]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7887
* Running on public URL: https://720281c09a7b01c957.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.


In [12]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("rohith", "1234"))

* Running on local URL:  http://127.0.0.1:7889
* To create a public link, set `share=True` in `launch()`.


## Forcing dark mode

In [13]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7890
* To create a public link, set `share=True` in `launch()`.


In [14]:
# Adding a little more:

message_input = gr.Textbox(label="Your message:", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=shout,
    title="Shout", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7891
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input hello


In [16]:
# And now - changing the function from "shout" to "message_gpt"

message_input = gr.Textbox(label="Your message:", info="Enter a message for GLM-4.5-Air", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=message_glm,
    title="GLM", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.


In [17]:
system_message = "You are a helpful assistant that responds in markdown without code blocks"

message_input = gr.Textbox(label="Your message:", info="Enter a message for GLM-4.5-Air", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=message_glm,
    title="GLM", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7894
* To create a public link, set `share=True` in `launch()`.


In [18]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python guide in the guides folder

def stream_glm(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = zai.chat.completions.create(
        model='GLM-4.5-Air',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [19]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GLM-4.5-Air", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_glm,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7895
* To create a public link, set `share=True` in `launch()`.


In [20]:
def stream_gemini(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = gemini.chat.completions.create(
        model='gemini-2.5-flash',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [21]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GLM-4.5-Air", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_glm,
    title="Claude", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7896
* To create a public link, set `share=True` in `launch()`.


## And now getting fancy

In [22]:
def stream_model(prompt, model):
    yield ""
    if model=="GLM":
        result = stream_glm(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [23]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["GLM", "Gemini"], label="Select model", value="GLM")
message_output = gr.Markdown(label="Response:")

# Build interface
view = gr.Interface(
    fn=stream_model,
    title="LLMs",
    inputs=[message_input, model_selector],
    outputs=message_output,
    examples=[
        ["Explain the Transformer architecture to a layperson", "GLM"],
        ["Explain the Transformer architecture to an aspiring AI engineer", "Gemini"]
    ],
    flagging_mode="never"
)

# Launch app
view.launch()

* Running on local URL:  http://127.0.0.1:7897
* To create a public link, set `share=True` in `launch()`.
